<a href="https://colab.research.google.com/github/Defconn4/cs499-final-project/blob/main/cs499_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

In [ ]:
# Clone github repository setup
# import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join  

# path to your project on Google Drive (where to save)
MY_GOOGLE_DRIVE_PATH = 'MyDrive/Github/cs499-final-project' 

# ***************** Replace with YOUR Github username *****************
GIT_USERNAME = "{INSERT YOUR GITHUB USERNAME HERE}"

# ***************** Replace with YOUR Github Personal Access Token *****************
GIT_TOKEN = "{INSERT YOUR TOKEN HERE}"

# Replace with your github repository
GIT_REPOSITORY = "cs499-final-project" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"    

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

In [ ]:
%cd "{PROJECT_PATH}"    # Change directory to the location defined in project_path
!git clone "{GIT_PATH}" # clone the github repository

In [ ]:
"""
  Begin actual Google Colab implementation - training the GPT-2 model.
"""
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import tensorflow as tf

In [ ]:
# Download GPT-2 model - we will be using the "small" default model, which is 500 MB on disk.
gpt2.download_gpt2(model_name="124M")
gpt2.mount_gdrive()                 # Mount model for data input from my Google Drive
model_downloaded_mounted = True

In [ ]:
# Upload files to the Colab notebook.
uploaded = files.upload()

Saving $uicideboy$.txt to $uicideboy$ (1).txt
Saving 6ix9ine.txt to 6ix9ine (1).txt
Saving 6lack.txt to 6lack (1).txt
Saving 21 Savage.txt to 21 Savage (1).txt
Saving A$AP Ferg.txt to A$AP Ferg (1).txt
Saving A$AP Rocky.txt to A$AP Rocky (1).txt
Saving Andre 3000.txt to Andre 3000 (1).txt
Saving B.o.B.txt to B.o.B (1).txt
Saving Baby Keem.txt to Baby Keem (1).txt
Saving Big Sean.txt to Big Sean (1).txt
Saving Chance The Rapper.txt to Chance The Rapper (1).txt
Saving Chief Keef.txt to Chief Keef (1).txt
Saving Childish Gambino.txt to Childish Gambino (1).txt
Saving Chris Brown.txt to Chris Brown (1).txt
Saving Comethazine.txt to Comethazine (1).txt
Saving Cordae.txt to Cordae (1).txt
Saving Dababy.txt to Dababy (1).txt
Saving Denzel Curry.txt to Denzel Curry (1).txt
Saving Doja Cat.txt to Doja Cat (1).txt
Saving Don Toliver.txt to Don Toliver (1).txt
Saving Dr.Dre.txt to Dr.Dre (1).txt
Saving Drake.txt to Drake (1).txt
Saving Eminem.txt to Eminem (1).txt
Saving Fivio Foreign.txt to Fivi

In [ ]:
"""
  Probe user for artist lyrics to generate.
  - Need to first manually upload files in Google Colab, which will just be each
    individual file from the .\lyrics\ directory on my desktop.
  - Then I neeed to use files.upload() to grab the files I've uploaded, where
    `uploaded` is a dictionary of keys = the encoded file object and
    values = the encoded file objects.
"""
def user_select_artist():
    user_artist = None

    # Search Google Drive for the file containing that artist's name, if it doesn't exist, prompt the user again.
    while user_artist == None:
        user_artist = input("Please type the full name (proper spelling and capitalization) of the rapper you wish to generate lyrics in the spirit of: ")
        
        # Try to find the key (artist name) in `uploaded`, if not there, ask the user again.
        try:
          key = user_artist + '.txt'
          file_lyrics = uploaded[key]
          print("CoZZy will learn to rap like : <%s>" % user_artist)
          break

        except Exception as e:
          print("CoZZy cannot find a rapper with name <%s> in his memory, please type a different rapper..." % user_artist, "\n\n")
          user_artist = None
    
    return user_artist, key

In [ ]:
# Grab user selected artist 
user_artist, artist_lyric_file_path = user_select_artist()
print("artist lyric file path: ", artist_lyric_file_path)

# Navigate to the directory containing the `artist_lyric_file_path`
lyrics_path = '/content/drive/MyDrive/Github/cs499-final-project/cs499-final-project/lyrics/' + artist_lyric_file_path
%cd '/content/drive/MyDrive/Github/cs499-final-project/cs499-final-project/lyrics'

# Move the lyrics file from the current directory that we navigated to above
# to /content/drive/MyDrive/
%cp "{lyrics_path}" "/content/drive/MyDrive"

# Copy file to the Colab VM from the `lyrics` directory.
gpt2.copy_file_from_gdrive(artist_lyric_file_path)

In [ ]:
"""
    Create a persistent TensorFlow session which stores the training configuration,
    and then run the training for the specified number of steps (to run indefinitely, step == -1).

    Model checkpoints are saved in \checkpoint\run1 by default and checkpoints are saved every 500 steps.
"""
session = gpt2.start_tf_sess()
tf.compat.v1.reset_default_graph()
if not session:
    session = gpt2.start_tf_sess()
else:
    session = gpt2.reset_session(session)

In [ ]:
print("CoZZy is learning to write his lyrics...\n")

# Finetune GPT-2 Model.
# run_name = subfolder within checkpoint to save the model (useful when working with multiple models)
gpt2.finetune(session,
              dataset = artist_lyric_file_path,
              model_name = '124M',
              steps = 1000,
              learning_rate = 0.0001,
              restore_from ='latest',           # Restore from latest checkpoint
              run_name ='run1',
              print_every = 1,
              sample_every = 200,
              save_every = 500
              )

CoZZy is learning to write his lyrics...

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  5.09it/s]


dataset has 25405 tokens
Training...
[1 | 102.40] loss=3.49 avg=3.49
[2 | 200.37] loss=3.47 avg=3.48
[3 | 298.00] loss=3.34 avg=3.43
[4 | 394.49] loss=3.40 avg=3.42
[5 | 491.55] loss=2.95 avg=3.33
[6 | 588.67] loss=2.75 avg=3.23
[7 | 685.37] loss=2.75 avg=3.16
[8 | 782.15] loss=2.87 avg=3.12
[9 | 879.29] loss=2.66 avg=3.07
[10 | 975.77] loss=2.86 avg=3.05
[11 | 1072.44] loss=2.87 avg=3.03
[12 | 1169.21] loss=2.35 avg=2.97
[13 | 1266.19] loss=2.70 avg=2.95
[14 | 1362.58] loss=2.76 avg=2.93
[15 | 1459.62] loss=2.67 avg=2.91
[16 | 1556.07] loss=2.29 avg=2.87
[17 | 1652.67] loss=2.06 avg=2.82
[18 | 1749.60] loss=2.32 avg=2.79
[19 | 1846.17] loss=2.52 avg=2.77
[20 | 1942.64] loss=2.59 avg=2.76
======== SAMPLE 1 ========


[Outro: Danny DeVito, Keanu Reeves, Robyn Hitchcock]

This is my first week on tour, and it's been one hell of a ride for me, I know you're watching and thinking about me, all this negativity that you've been under that your sister told her momma told her daddy about. You 

In [ ]:
# After the model is done training, copy the checkpoint folder to your my Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Copy the checkpoint to the Google drive.
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [ ]:
# The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.
# IMPORTANT NOTE: If you want to rerun this cell, restart the VM first (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

# session = gpt2.start_tf_sess()
tf.compat.v1.reset_default_graph()
if not session:
    session = gpt2.start_tf_sess()
else:
    session = gpt2.reset_session(session)
    
gpt2.load_gpt2(session, run_name='run1')

Loading checkpoint checkpoint/run1/model-100
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-100


In [ ]:
"""
  After training the model / loading the retrained model from a checkpoint, we can now generate text!

  You can also generate multiple texts at a time by specifing nsamples.
  Unique to GPT-2, you can pass a batch_size to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for batch_size).
  Other optional-but-helpful parameters for gpt2.generate and friends:

  length: Number of tokens to generate (default 1023, the maximum)
  temperature: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
  top_k: Limits the generated guesses to the top k guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set top_k=40)
  top_p: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with top_p=0.9)
  truncate: Truncates the input text until a given sequence, excluding that sequence (e.g. if truncate='<|endoftext|>', the returned text will include everything before the first <|endoftext|>). It may be useful to combine this with a smaller length if the input texts are short.
  include_prefix: If using truncate and include_prefix=False, the specified prefix will not be included in the returned text.
"""

# For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer.
# This cell will generate a generated text file with a unique timestamp - you can rerun the cells as many times as you want for even more generated texts!
from pytz import timezone

est = timezone('EST')
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.now(est))
gpt2.generate(session,
              destination_path = gen_file,
              length = 250,
              temperature = 0.7,
              prefix = "RAP",
              truncate = '<|EndSong|>',
              nsamples = 5,
              batch_size = 1,
              run_name='run1'
              )

In [ ]:
# You may have to run this cell twice to get the file to download.
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Calculate a Bleu score relative to generated text and the training file of `artist_name.txt`.
!pip install --upgrade bleu
from bleu import file_bleu
import shutil
import os

# Make a copy of the file present in `artist_lyric_file_path`.
shutil.copy(artist_lyric_file_path, './copy.txt')

# Find number of lines in gen_file.
with open(gen_file, 'r') as f:
  gen_file_lines = len(f.readlines())
  print('Total lines:', gen_file_lines)

# Truncate ./copy.txt to same number gen_file_lines.
with open('./copy.txt', 'r+') as c:
  lines = c.readlines()
  c.truncate()
  lines = lines[:gen_file_lines]
  c.close()

# Write the same number of lines from above for evaluation.
with open('./copy.txt', 'w') as file_copy:
  file_copy.writelines(lines)
  file_copy.close()

# Success! Bleu score: 25.32
bleu_score = file_bleu(gen_file, './copy.txt')
print("Bleu Score between generated text and lyric file for selected artist: ", bleu_score)

In [ ]:
# When we want to commit changes to our repo (ran in different cells to ensure
# everything works properly).
# %cd {PROJECT_PATH}

In [ ]:
# Check status of changed files in directory
# !git status

In [ ]:
# Add / track changed files
# !git add .

In [ ]:
# Commit the changes
# !git commit -m "MESSAGE HERE"
# !git config --global user.email "{YOUR_EMAIL}"
# !git config --global user.name "{YOUR_GITHUB_USERNAME}"

In [ ]:
# Push changes to repo
# !git push origin main